In [44]:
import networkx as nx
import sys
sys.path.append('..')
from dataset import load_graph
import random
import numpy as np
from gensim.models import Word2Vec
from timeit import default_timer as timer


class node2vec(object):
    def __init__(self,config):
        self.graph=load_graph(config['graph_path'],weight=False).to_undirected()
        self.r=config['r']
        self.walk_length=config['walk_length']
        self.dim=config['dim']
        self.p=config['p']
        self.q=config['q']
        self.window=config['window']
        
        
    def walking(self):
        walks=[]
        nodes=list(self.graph.nodes)
        for i in range(self.r):
            random.shuffle(nodes)
            for n in nodes:
                walks.append(self.node2vec_walk(n))
        return walks
                
    def node2vec_walk(self,src):
        walk=[src]
        while len(walk) < self.walk_length:
            cur_node=walk[-1]
            cur_node_neigh=sorted(self.graph.neighbors(cur_node))
            if len(cur_node_neigh)>0:
                # For walking at first
                if len(walk)==1:
                    walk.append(cur_node_neigh[self.direct_to_node(self.alias_nodes[cur_node][0],self.alias_nodes[cur_node][1])])
                else:
                    pre_node=walk[-2]
                    next_node=cur_node_neigh[self.direct_to_node(self.alias_edges[(pre_node,cur_node)][0],self.alias_edges[(pre_node,cur_node)][1])]
                    walk.append(next_node)
            else:
                break
        return walk    
    
    def preprocess_Modified(self):
        # Weight Setting
        for edge in self.graph.edges():
            self.graph[edge[0]][edge[1]]['weight']=1
            self.graph[edge[1]][edge[0]]['weight']=1
            
        alias_nodes={}
        alias_edges={}
        
        # For alias sampling
        # alias nodes would be used when walking starts at first
        for node in self.graph.nodes():
            node_prob=[self.graph[node][n]['weight'] for n in sorted(self.graph.neighbors(node))]
            Z=float(sum(node_prob))
            normalize_prob=[e/Z for e in node_prob]
            alias_nodes[node]=self.alias_setting(normalize_prob)
            
        # For alias sampling 
        # alias edge would be used after the walking at first
        # Undirected Graph
        for edge in self.graph.edges():
            alias_edges[edge]=self.get_alias_edge(edge[0],edge[1])
            alias_edges[(edge[1],edge[0])]=self.get_alias_edge(edge[1],edge[0])
            
        self.alias_nodes=alias_nodes
        self.alias_edges=alias_edges
    
    
    def get_alias_edge(self,src,dst):
        """
        For preprocess the transition probability
        """
        edge_prob=[]
        for dst_neighbor in sorted(self.graph.neighbors(dst)):
            # distance : 0
            if dst_neighbor==src:
                edge_prob.append(self.graph[dst][dst_neighbor]['weight']/self.p)
            # distance : 1
            elif self.graph.has_edge(dst_neighbor,src):
                edge_prob.append(self.graph[dst][dst_neighbor]['weight'])
            # distance : 2
            else:
                edge_prob.append(self.graph[dst][dst_neighbor]['weight']/self.q)
        Z=float(sum(edge_prob))
        norm_prob=[e/Z for e in edge_prob]
        return self.alias_setting(norm_prob)    
    
    def alias_setting(self,prob):
        """
        Alias Method
        reference : https://en.wikipedia.org/wiki/Alias_method (table generation)
        At Last, U_i is filled with value of smaller than 1
        """
        n=len(prob)
        U=np.zeros(n)
        # for floor the value
        K=np.zeros(n, dtype=np.int)
        overfull=[]
        underfull=[]
        for i,p in enumerate(prob):
            U[i]=n*p
            if U[i]<1.0:
                underfull.append(i)
            else:
                overfull.append(i)
                
        while len(overfull)>0 and len(underfull)>0:
            under=underfull.pop()
            over=overfull.pop()
            K[under]=over
            U[over]+=U[under]-1.0
            if U[over]<1.0:
                underfull.append(over)
            else:
                overfull.append(over)
        return K,U
            
        
    def direct_to_node(self,K,U):
        """
        Direct to next node using alias sampling 
        """
        n=len(K)
        node=int(np.floor(np.random.rand()*n))
        return node if np.random.rand()<U[node] else K[node]
    def make_string(self,walks):
        walk=[]
        for i in walks:
            temp=[]
            for j in i:
                temp.append(str(j))
            walk.append(temp)
        return walk
    
    @staticmethod
    def run(graph):
        start=timer()
        print("Preprocess Start")
        graph.preprocess_Modified()
        print("Preprocess Finish, Time :{:.4f}".format(timer()-start))
        start=timer()
        walks=graph.walking()
        print("Walking Finish, Time : {:.4f}".format(timer()-start))
#         flatten = lambda l: [item for sublist in l for item in sublist]
#         walks=[map(str,i) for i in walks]
        walks=graph.make_string(walks)
        start=timer()
        model=Word2Vec(sentences=walks, size=graph.dim,window=graph.window,min_count=0,sg=1,workers=8,iter=1)
        print("SGD Finish, Time : {:.4f}".format(timer()-start))
        return model.wv
    


In [53]:
from dataset import load_data,load_graph
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import Pipeline

class Prediction(object):
    def __init__(self,feature_vector):
        self.vector=feature_vector
        self.adj, _,_,_,_,_,_,_,label=load_data()
        self.label=label.cpu()
        self.graph=load_graph().to_undirected()
        
    def link_prediction_classifier(self,max_iter=2000):
        lr_clf = LogisticRegressionCV(Cs=10, cv=10, scoring="roc_auc", max_iter=max_iter)
        return Pipeline(steps=[("sc", StandardScaler()), ("clf", lr_clf)])    
    
    def evaluate_link_prediction(self):
        
    
    def load_label(path='../data/ind.cora.y'):
        label=None
        with open(path,'rb') as file:
            label=pickle.load(file)
        return label
    
    
    def train_link_prediction_model(self):
        clf=self.link_prediction_classifier()
        link_feature,label=self.link_example_to_feature()
        clf.fit(link_feature,label)
        return clf
        
    def link_example_to_feature(self):
        link_feature=[]
        label=[]
        node=sorted(list(self.graph.nodes()))
        for index,node_1 in enumerate(node):
            for node_2 in node[index,:]:
                if node_1 == node_2:
                    continue
                if self.graph.has_edge(node_1,node_2):
                    label.append(1)
                    link_feature.append(self.binary_operator(node_1,node_2))
                else:
                    label.append(0)
                    link_feature.append(self.binary_operator(node_1,node_2))
        train_link
        return link_feature,label
                
    
    def binary_operator(self,src,dst):
        return np.asarray(self.vector[str(src)])*np.asarray(self.vector[str(dst)])
        

In [14]:
config={'r':10,
        'walk_length':80,
        'dim':128,
        'p':1,
        'q':1,
        'graph_path':'../data/ind.cora',
        'window':10
}

In [45]:
graph=node2vec(config)

In [46]:
vector=node2vec.run(graph)

Preprocess Start
Preprocess Finish, Time :0.3184
Walking Finish, Time : 8.6896
SGD Start
SGD Finish, Time : 5.8401


In [66]:
vector['0']

array([-6.83663785e-02,  1.07074723e-01, -1.45433471e-01,  7.72097111e-02,
       -1.74917608e-01,  1.48178667e-01, -4.80675519e-01,  9.81525630e-02,
       -2.07178250e-01,  5.25613278e-02,  5.82001843e-02, -3.94587934e-01,
        2.39404395e-01, -2.18963306e-02,  3.25878225e-02, -1.46705374e-01,
        5.22884846e-01, -2.51476496e-01,  5.97268939e-01, -4.78149831e-01,
        5.11971302e-03, -1.02224583e-02,  4.55449313e-01, -2.87113577e-01,
       -3.33695225e-02, -7.52046183e-02, -1.08576603e-02, -1.98904037e-01,
       -1.93848670e-01,  1.85810596e-01,  7.40599692e-01, -6.85704648e-01,
       -2.58348197e-01,  2.74352878e-01,  2.69702345e-01,  2.45061591e-01,
       -2.61042178e-01, -3.05080295e-01, -5.61426058e-02,  6.17093071e-02,
       -4.75103438e-01, -3.52188528e-01,  1.94744438e-01,  4.83613938e-01,
        7.53225153e-03, -5.62273026e-01,  1.16552167e-01,  6.32723093e-01,
       -4.51847725e-02, -1.13065034e-01,  1.85359836e-01, -3.47874761e-01,
       -5.90517104e-01, -